In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import glob, os

import sys

module_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)

module_dir = os.path.abspath(os.path.join(os.getcwd(), '../../sdt_dask/'))
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)

module_dir = os.path.abspath(os.path.join(os.getcwd(), '../../solardatatools/'))
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)

module_dir = os.path.abspath(os.path.join(os.getcwd(), '../dask_tool/'))
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)
    
from sdt_dask.dask_tool.sdt_dask import SDTDask

In [4]:
SDTDask?

## Local Client example 
 use local csv dataplug and local client

### Instantiate a localfile data plug

In [ ]:
from sdt_dask.dataplugs.csv_plug import LocalFiles

In [ ]:
LocalFiles?

In [ ]:
path = "../dataplugs/example_data/"
local_file_data_plug = LocalFiles(path_to_files=path)
local_file_keys = [(os.path.basename(fname)[:-4],) for fname in glob.glob(path + "*")]
local_file_keys

In [ ]:
local_file_data_plug.get_data(local_file_keys[1])

### Set a local User-defined Client

In [ ]:
from dask.distributed import Client

In [ ]:
n_workers = 1
threads_per_worker = 1
total_system_memory = 8
memory_per_worker = total_system_memory / n_workers
local_client = Client(processes=False, memory_spill_fraction=False, memory_pause_fraction=False, memory_target_fraction=0.8, n_workers=n_workers, threads_per_worker=threads_per_worker, memory_limit=f"{memory_per_worker}GiB")
local_client.dashboard_link

In [ ]:
dask_tool = SDTDask(local_file_data_plug, local_client)

dask_tool.execute(local_file_keys, fix_shifts=True, verbose=True)

## AWS Fargate Client example
use LocalFile dataplug and fargate client

### Instantiate a pvdaq data plug


In [5]:
from sdt_dask.dataplugs.pvdaq_plug import PVDAQPlug

In [6]:
PVDAQPlug?

In [7]:
pvdaq_data_plug = PVDAQPlug()

### Set up a Fargate cluster client


In [8]:
from sdt_dask.clients.aws.fargate import Fargate

In [9]:
# Use should define an environment variable for secret key
PA_NUMBER = os.getenv("project-pa-number")
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')
ENVIRONMENT = {
    'AWS_ACCESS_KEY_ID' : os.getenv('AWS_ACCESS_KEY_ID'),
    'AWS_SECRET_ACCESS_KEY' : os.getenv('AWS_SECRET_ACCESS_KEY')
}

In [10]:
# The Tag, VPC, image, workers, threads per worker and environment need to be user defined and passed to the client class
TAGS = {
    "project-pa-number": PA_NUMBER,
    "project": "pvinsight"
}
VPC = "vpc-ab2ff6d3" # for us-west-2
IMAGE = "smiskov/dask-sdt-sm-2:latest"

WORKERS = 3
THREADS_PER_WORKER = 1

In [ ]:
# print(TAGS, ENVIRONMENT)

In [11]:
fargate_client = Fargate().init_client(image=IMAGE, 
                               tags=TAGS, 
                               vpc=VPC, 
                               region_name=AWS_DEFAULT_REGION,
                               environment=ENVIRONMENT,
                               n_workers=WORKERS,
                               threads_per_worker=THREADS_PER_WORKER
                               )

[i] Initilializing Fargate Cluster ...


C:\Users\Zander\anaconda3\envs\pvi-user\lib\contextlib.py:142: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)


[i] Initilializing Dask Client ...
[>] Dask Dashboard: http://35.94.98.144:8787/status


In [12]:
pvdaq_keys = [(34, 2011), (35, 2015), (51,2012)]
dask_tool = SDTDask(pvdaq_data_plug, fargate_client)

dask_tool.execute(pvdaq_keys, fix_shifts=True, verbose=True)

RuntimeError: Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments


## Azure Client example